## Cross Validation : 교차검증

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [2]:
iris =load_iris()
features = iris['data']
label = iris['target']

In [3]:
# 불류기 객체 생성

dt_clf = DecisionTreeClassifier(random_state = 11)

In [4]:
# 5개의 폴드로 구분하고 각각의 accuracy를 저장할 리스트 생성

kfold = KFold(n_splits = 5)
cv_accu = []

In [5]:
print(f'data set의 크기 : {features.shape}')

data set의 크기 : (150, 4)


In [6]:
n_iter = 0

# KFold의 split()은 폴드 별로 학습용과, 검증용 데이터의 로우 인덱스를 array로 반환

for train_index , val_index in kfold.split(features):
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = label[train_index], label[val_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_val)
    
    #반복시마다 accuracy 측정
    accuracy = np.round(accuracy_score(y_val, pred), 3)
    n_iter += 1
    
    train_size = X_train.shape[0]
    val_size = X_val.shape[0]
    print(f'\n# {n_iter}차 교차검증 : 정확도 {accuracy}, 학습 데이터 크기 {train_size}, 검증 데이터 크기 {val_size}\n')
    
    print(f'검증세트 인덱스 : {val_index}\n')
    
    cv_accu.append(accuracy)
    
# for 문 종료 후 정확도의 평균 계산

print(f'평균 accuracy : {np.mean(cv_accu)}')


# 1차 교차검증 : 정확도 1.0, 학습 데이터 크기 120, 검증 데이터 크기 30

검증세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


# 2차 교차검증 : 정확도 0.967, 학습 데이터 크기 120, 검증 데이터 크기 30

검증세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]


# 3차 교차검증 : 정확도 0.867, 학습 데이터 크기 120, 검증 데이터 크기 30

검증세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]


# 4차 교차검증 : 정확도 0.933, 학습 데이터 크기 120, 검증 데이터 크기 30

검증세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]


# 5차 교차검증 : 정확도 0.833, 학습 데이터 크기 120, 검증 데이터 크기 30

검증세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

평균 accuracy : 0.9200000000000002


In [7]:
# 각 label별 동일한 비율로 학습시키기를 원하면 StratifiedKFold를 이용하면 된다
# from sklearn.model_selection import StratifiedKFold

# 주의할 점은 for 문에서 label세트도 추가로 넣어 돌려 줘야 한다는 점
# ex) for train_index, val_index in skfold.split(features, label):

In [8]:
# 이러한 일련의 과정을 보다 간단히 : cross_val_score()

from sklearn.model_selection import cross_val_score, cross_validate

iris = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 11)
features = iris.data
label = iris.target

# 성능지표는 accuracy, 교차검증 세트는 5개
scores = cross_val_score(dt_clf, features, label, scoring = 'accuracy', cv = 5)

print(f'교차 검증별 정확도 : {np.round(scores, 4)}')
print(f'평균 검증 정확도 : {np.round(np.mean(scores), 4)}')

# cross_val_score는 내부적으로 stractified validation을 이용한다.
# accuracy이외의 성능 지표와 수행 시간을 알기 위해서는 cross_validate()를 이용한다.

교차 검증별 정확도 : [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도 : 0.96


## 데이터 전처리 중 레이블 인코딩

In [9]:
from sklearn.preprocessing import LabelEncoder

mag7 = ['엔비디아', '마이크로소프트', '애플', '구글', '메타', '아마존','테슬라']

# LabelEncoder을 객체로 생성 후, fit()과 transform()으로 레이블 인코딩 수행
encoder = LabelEncoder()
encoder.fit(mag7)
labels = encoder.transform(mag7)

print(f'변환값 : {labels}')

print(f'인코딩 클래스 : {encoder.classes_}')

변환값 : [5 1 4 0 2 3 6]
인코딩 클래스 : ['구글' '마이크로소프트' '메타' '아마존' '애플' '엔비디아' '테슬라']


In [10]:
print(f'디코딩 원본값 : {encoder.inverse_transform([0,1,2,3,4,5,6])}')

디코딩 원본값 : ['구글' '마이크로소프트' '메타' '아마존' '애플' '엔비디아' '테슬라']


## 데이터 전처리 중 feature 인코딩 : pandas의 get_dummies()와 유사

In [11]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

mag7 = ['엔비디아', '마이크로소프트', '애플', '구글', '메타', '아마존','테슬라']

# 2차원의 데이터(ndarray로 전환)

mag7 = np.array(mag7).reshape(7,1)

mag7

array([['엔비디아'],
       ['마이크로소프트'],
       ['애플'],
       ['구글'],
       ['메타'],
       ['아마존'],
       ['테슬라']], dtype='<U7')

In [12]:
#OneHotEncoding을 실시

oh_encoder = OneHotEncoder()
oh_encoder.fit(mag7)
oh_features = oh_encoder.transform(mag7)
print(oh_features)


  (0, 5)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 0)	1.0
  (4, 2)	1.0
  (5, 3)	1.0
  (6, 6)	1.0


In [13]:
# 다시 array로 전환

print(oh_features.toarray())

print(oh_features.shape)

[[0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
(7, 7)


In [17]:
# pandas를 이용한 one hot encoding

import pandas as pd

mag7 = ['엔비디아', '마이크로소프트', '애플', '구글', '메타', '아마존','테슬라']

df = pd.DataFrame({'company' : mag7})
pd.get_dummies(df)

,company_구글,company_마이크로소프트,company_메타,company_아마존,company_애플,company_엔비디아,company_테슬라
0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0
5,0,0,0,1,0,0,0
6,0,0,0,0,0,0,1


## Feature Scaling

In [28]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

df = pd.DataFrame(data = iris['data'], columns = iris['feature_names'])

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [29]:
print(df.mean())
print()
print(df.std())

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

sepal length (cm)    0.828066
sepal width (cm)     0.435866
petal length (cm)    1.765298
petal width (cm)     0.762238
dtype: float64


### Standard Scaler : mean 0, std 1 ( z 변환)
### Min Max Scaler : (x - min) / (max - min)

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scaler 객체 생성
s_scaler = StandardScaler()
m_scaler = MinMaxScaler()

#fit()과 transform()으로 변환 
s_scaler.fit(df)
s_scaled = s_scaler.transform(df)

m_scaler.fit(df)
m_scaled = m_scaler.transform(df)

# transform으로 변환시 numpy array가 되므로 다시 DataFrame으로 변환
s_scaled_df = pd.DataFrame(data = s_scaled, columns = iris['feature_names'])
m_scaled_df = pd.DataFrame(data = m_scaled, columns = iris['feature_names'])

In [31]:
print(f'standard scaler 변환 후의 값들')
print(s_scaled_df.mean())
print(s_scaled_df.std())
print(s_scaled_df.min())
print(s_scaled_df.max())

standard scaler 변환 후의 값들
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
sepal length (cm)    1.00335
sepal width (cm)     1.00335
petal length (cm)    1.00335
petal width (cm)     1.00335
dtype: float64
sepal length (cm)   -1.870024
sepal width (cm)    -2.433947
petal length (cm)   -1.567576
petal width (cm)    -1.447076
dtype: float64
sepal length (cm)    2.492019
sepal width (cm)     3.090775
petal length (cm)    1.785832
petal width (cm)     1.712096
dtype: float64


In [33]:
print(f'min max scaler 변환 후의 값들')
print(m_scaled_df.mean())
print(m_scaled_df.std())
print(m_scaled_df.min())
print(m_scaled_df.max())

min max scaler 변환 후의 값들
sepal length (cm)    0.428704
sepal width (cm)     0.440556
petal length (cm)    0.467458
petal width (cm)     0.458056
dtype: float64
sepal length (cm)    0.230018
sepal width (cm)     0.181611
petal length (cm)    0.299203
petal width (cm)     0.317599
dtype: float64
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64
